In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('./Invitation Addresses - RSVPs.csv')

In [3]:
df = df.dropna(how='all').reset_index()

In [4]:
df.shape

(174, 14)

In [5]:
df.head()

,index,Table,Last Name,First Name,Title,Beef,Chicken,Vegetarian,Special Meal,Chicken Fingers,Under 21/Vendor Meal,Unnamed: 10,Total Attending,174
0,0,12,Ahmed,Aneela,Ms.,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,8,Bailey,Jody,Mrs.,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,8,Bailey,Budd,Mr.,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,12,Bari,Omar,Dr.,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,8,Bateman,HM,Ms.,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
def get_meal(row):
    meal = (row[['Beef','Chicken','Vegetarian','Chicken Fingers']]
            .dropna()
            .index[0])
    
    if meal == 'Chicken Fingers':
        return 'Chicken Fingers / Under 21'
    else:
        return meal

In [8]:
from jinja2 import Environment, FileSystemLoader
import os
THIS_DIR = os.path.dirname(os.path.abspath("template.html"))

j2_env = Environment(loader=FileSystemLoader(THIS_DIR),
                         trim_blocks=True)

In [9]:
template = j2_env.get_template('template.html')

In [10]:
pages = []
cur_page = []
for idx, row in df.iterrows():
    
    if idx % 4 == 0 and idx != 0:
        pages.append(cur_page)
        cur_page = []
    
    if row['Table'] == 'Sweetheart':
        table='Sweetheart Table'
    elif row['Table'] == 'Vendor':
        table='Vendor Table'
    else:
        table='Table ' + str(row['Table'])
    
    meal = get_meal(row)
    
    cur_page.append([' '.join([row['First Name'], '<br/>', row['Last Name']]),table, meal])
    
    
    if idx == len(df) - 1:
        pages.append(cur_page)

In [11]:
pages

[[['Aneela <br/> Ahmed', 'Table 12', 'Beef'],
  ['Jody <br/> Bailey', 'Table 8', 'Beef'],
  ['Budd <br/> Bailey', 'Table 8', 'Beef'],
  ['Omar <br/> Bari', 'Table 12', 'Beef']],
 [['HM <br/> Bateman', 'Table 8', 'Chicken'],
  ['Olivia <br/> Bobrowsky', 'Table 11', 'Beef'],
  ['Maryann <br/> Bochenek', 'Table 17', 'Beef'],
  ['Camille <br/> Brandon', 'Table 3', 'Chicken']],
 [['Robert <br/> Brandon', 'Table 3', 'Beef'],
  ['Peter <br/> Brunner', 'Table 2', 'Chicken'],
  ['Carol <br/> Bubar', 'Table 18', 'Beef'],
  ['David <br/> Bubar', 'Table 18', 'Chicken']],
 [['Elizabeth <br/> Buckley', 'Table 8', 'Beef'],
  ['Martin <br/> Burke', 'Table 14', 'Beef'],
  ['Timothy <br/> Cashmore', 'Table 8', 'Beef'],
  ['Patricia <br/> Catanzaro', 'Table 7', 'Chicken']],
 [['Joseph <br/> Catanzaro', 'Table 7', 'Beef'],
  ['Joey <br/> Catanzaro', 'Table 10', 'Beef'],
  ['Judith <br/> Chaifetz', 'Table 9', 'Beef'],
  ['Malcolm <br/> Chaifetz', 'Table 9', 'Beef']],
 [['Phoebe Li <br/> Chen', 'Table 15', 

In [28]:
%%time
def get_fields(page):
    name_1, table_1, name_2, table_2, name_3, table_3, name_4, table_4, meal_1, meal_2, meal_3, meal_4 = '', '', '', '', '', '', '', '', '', '', '', ''
    
    try:
        name_1, table_1, meal_1 = page[0][0], page[0][1], page[0][2]
    except:
        pass
    
    try:
        name_2, table_2, meal_2 = page[1][0], page[1][1], page[1][2]
    except: 
        pass
    
    try:
        name_3, table_3, meal_3 = page[2][0], page[2][1], page[2][2]
    except:
        pass
    
    try:
        name_4, table_4, meal_4 = page[3][0], page[3][1], page[3][2]
    except:
        pass
    
    return (name_1, table_1, meal_1, 
            name_2, table_2, meal_2,
            name_3, table_3, meal_3,
            name_4, table_4, meal_4)
    
for idx, page in enumerate(pages):
    (name_1, table_1, meal_1, 
            name_2, table_2, meal_2,
            name_3, table_3, meal_3,
            name_4, table_4, meal_4) = get_fields(page)
    
    html = template.render(
       name_1=name_1, 
        table_1=table_1, 
        name_2=name_2, 
        table_2=table_2, 
        name_3=name_3, 
        table_3=table_3, 
        name_4=name_4, 
        table_4=table_4,
        meal_1=meal_1,
        meal_2=meal_2,
        meal_3=meal_3,
        meal_4=meal_4
    )
    
    with open('./test-{}.html'.format(idx),'w') as otp:
        otp.writelines(html)
    
    print('Creating PDF... {} of {}'.format(idx+1, len(pages)))
    os.system('electron-pdf test-{:02d}.html test-{:02d}.pdf'.format(idx,idx))
    
#     if idx > 2: break
    
os.system('gs -q -dNOPAUSE -dBATCH -sDEVICE=pdfwrite -sOutputFile=merged.pdf test-*.pdf')
os.system('mv test-* unused/')

Creating PDF... 1 of 44
Creating PDF... 2 of 44
Creating PDF... 3 of 44
Creating PDF... 4 of 44
Creating PDF... 5 of 44
Creating PDF... 6 of 44
Creating PDF... 7 of 44
Creating PDF... 8 of 44
Creating PDF... 9 of 44
Creating PDF... 10 of 44
Creating PDF... 11 of 44
Creating PDF... 12 of 44
Creating PDF... 13 of 44
Creating PDF... 14 of 44
Creating PDF... 15 of 44
Creating PDF... 16 of 44
Creating PDF... 17 of 44
Creating PDF... 18 of 44
Creating PDF... 19 of 44
Creating PDF... 20 of 44
Creating PDF... 21 of 44
Creating PDF... 22 of 44
Creating PDF... 23 of 44
Creating PDF... 24 of 44
Creating PDF... 25 of 44
Creating PDF... 26 of 44
Creating PDF... 27 of 44
Creating PDF... 28 of 44
Creating PDF... 29 of 44
Creating PDF... 30 of 44
Creating PDF... 31 of 44
Creating PDF... 32 of 44
Creating PDF... 33 of 44
Creating PDF... 34 of 44
Creating PDF... 35 of 44
Creating PDF... 36 of 44
Creating PDF... 37 of 44
Creating PDF... 38 of 44
Creating PDF... 39 of 44
Creating PDF... 40 of 44
Creating 

In [22]:
# template.render(name_1="Amit Shavit", name_2="Amanda NeMoyer", table_1=5, table_2=10)